<a href="https://colab.research.google.com/github/24Priyanshi/Disaster-Resilience-/blob/main/xView2_Training_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloned Repository

In [ ]:
!git clone https://github.com/DIUx-xView/xView2_baseline.git
%cd xView2_baseline
!pip install -r requirements.txt
!pip install tensorflow opencv-python shapely geopandas tqdm


Cloning into 'xView2_baseline'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 165 (delta 18), reused 17 (delta 15), pack-reused 141 (from 1)
Receiving objects: 100% (165/165), 3.43 MiB | 3.53 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/xView2_baseline
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.0/130.0 kB 1.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


Mount Data/Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Defining dataset path

In [ ]:
RAW_DATA = "/content/drive/MyDrive/xBD_raw"
PROCESSED_DATA = "/content/drive/MyDrive/xBD"


Preprocessing dataset

In [ ]:
!python utils/split_into_disasters.py --input $RAW_DATA --output $PROCESSED_DATA
!python utils/mask_polygons.py --input $PROCESSED_DATA --single-file --border 2
!bash utils/data_finalize.sh -i $PROCESSED_DATA -x /content/xView2_baseline/ -s 0.75


Training the localized model

In [ ]:
%cd spacenet/src/models/
!python train_model.py \
  /content/drive/MyDrive/xBD/spacenet_gt/dataSet/ \
  /content/drive/MyDrive/xBD/spacenet_gt/images/ \
  /content/drive/MyDrive/xBD/spacenet_gt/labels/ \
  -e 100 -g 0


Process Data for Classification

In [ ]:
%cd /content/xView2_baseline/
!python model/process_data.py \
  --input_dir $PROCESSED_DATA \
  --output_dir $PROCESSED_DATA/processed_polygons/ \
  --output_dir_csv $PROCESSED_DATA/processed_polygons/ \
  --val_split_pct 0.2


Training the damage classification model

In [ ]:
!python model/damage_classification.py \
  --train_data $PROCESSED_DATA/processed_polygons/train \
  --train_csv $PROCESSED_DATA/processed_polygons/train.csv \
  --test_data $PROCESSED_DATA/processed_polygons/test \
  --test_csv $PROCESSED_DATA/processed_polygons/test.csv \
  --model_out $PROCESSED_DATA/classifier_weights


Run Inference

In [ ]:
!bash utils/inference.sh \
  -x /content/xView2_baseline/ \
  -i $PROCESSED_DATA/guatemala-volcano/images/guatemala-volcano_00000023_pre_disaster.png \
  -p $PROCESSED_DATA/guatemala-volcano/images/guatemala-volcano_00000023_post_disaster.png \
  -l $PROCESSED_DATA/localization_weights.h5 \
  -c $PROCESSED_DATA/classifier_weights.hdf5 \
  -o $PROCESSED_DATA/output.png -y
